In [17]:
import Pkg; Pkg.activate("../"); Pkg.instantiate()

  Activating project at `~/Kaleb_Data/PhD_Research/RoboSim.jl`
Precompiling project...
  ? Convex


In [3]:
using LinearAlgebra
using BlockDiagonals
using ForwardDiff
using ControlSystems
using ComponentArrays
using Parameters: @unpack
using StaticArrays
using Plots

In [4]:
# include dependencies
include("../src/utils.jl")
include("../src/ExpDynamicsLibrary.jl")
include("../src/RefTrajectoryLibrary.jl")
include("../src/ExpControllerLibrary.jl")
include("../src/Visualization.jl")
include("../src/RefTrajectoryLibrary.jl")

Precompiling Convex
        Info Given Convex was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
  ? Convex
[ Info: Precompiling Convex [f65535da-76fb-5f13-bab9-19810c17039a]
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
[ Info: Skipping precompilation since __precompile__(false). Importing Convex [f65535da-76fb-5f13-bab9-19810c17039a].


Main.RefTrajLib

In [5]:
include("../src/ExpEwareFlylab_v2.jl")

Main.EwareExpDIFL

In [6]:
# Testing Eware

# Initial state
soc_init = 30.0
N = 0.0
int_pos = [0.0; 0.0; 2.7]

# Get the quad_parameters
quad_p = ExpDyn.quadrotor_parameters

## Define the quad's initial condition
quad_ic = ComponentArray(
    x = int_pos,
    v = zeros(3),
    R = 1.0(I(3)) |> collect, 
    Ω = zeros(3),
    ω = 2*ExpDyn.hover_ω(quad_p),
    soc = soc_init
);

discharge_rate = 0.6667
T_SIM = 100.0

# params = EwareExpDI.simulate(quad_ic, 0., T_SIM, 1, discharge_rate)

sol_nc, params = EwareExpDIFL.simulate(quad_ic, 0., T_SIM, 1, discharge_rate)
X1 = sol_nc.u;
T = sol_nc.t;
# println("size of state vecs: $(size(sol_nc.u))")
# println("size of state vecs: $(size(sol_nc.t))")

=========== In recompute_committed ==============
Current time: 0.0
=========== Out recompute_committed ==============
=========== In recompute_committed ==============
Current time: 1.0
=========== Out recompute_committed ==============
=========== In recompute_committed ==============
Current time: 2.0
=========== Out recompute_committed ==============
=========== In recompute_committed ==============
Current time: 3.0
=========== Out recompute_committed ==============
=========== In recompute_committed ==============
Current time: 4.0
=========== Out recompute_committed ==============
=========== In recompute_committed ==============
Current time: 5.0
=========== Out recompute_committed ==============
=========== In recompute_committed ==============
Current time: 6.0
=========== Out recompute_committed ==============
=========== In recompute_committed ==============
Current time: 7.0
=========== Out recompute_committed ==============
=========== In recompute_committed =============

In [7]:
params[1].compute_time;

In [8]:
include("../src/RefTrajectoryLibrary.jl")

Main.RefTrajLib

In [9]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8700


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [12]:
X_bound = [[-3.60, -4.80, 1.0],
            [0.0, -4.80, 1.0],
            [2.40, -4.80, 1.0],
            [2.40, 0.0, 1.0],
            [2.40, 1.80, 1.0],
            [0.0, 1.80, 1.0],
            [-3.60, 1.80, 1.0],
            [-3.40, -1.40, 1.0]]

# Generate 100 values for z from 0 to 2.0
z_values = range(0, stop=8.0, length=400)

# Generate the 500-element vector (each with 100 points)
points = [[x, y, z] for (x, y, _) in X_bound for z in z_values];

Visualization.vis_traj!(vis, 2, :lab_bound, points; R = 0.01, color = MeshCat.RGBA(0.0, 1.0, 0.0, 2.5))

In [13]:
# [Quadrotors] Visualize the Quadrotors
Visualization.animate_system_geometric!(vis, X1, 1, 0.01)

In [14]:
# [Trajectories] Create and Visualize the reference trajectories
include("../src/RefTrajectoryLibrary.jl")
# For Quad 1
t0 = 0.
N = 4800
traj_type = 1
dt = 0.05

# Draw first trajectory
Xref_1, Uref1 = RefTrajLib.DI_3D_get_desired_trajectory(t0, traj_type, N, dt, 1); # The last argument is for the param type
Visualization.vis_traj!(vis, 1, :traj_1, Xref_1[1:1200]; R = 0.01, color = MeshCat.RGBA(1.0, 0.0, 0.0, 2.5))

In [15]:
# setobject!(vis, HyperRectangle(Vec(0., 0, 0), Vec(1., 1, 1)), MeshCat.MeshPhongMaterial(color = MeshCat.RGBA(0.0,1.0,0.0,0.2)))
# settransform!(vis, MeshCat.Translation(-0.5, -0.5, 0))
# # setprop!(vis, "material.opacity", 0.5)  # Correct usage of setprop!

## Trajectory Start point

In [146]:
trajectory_params_0 = ComponentArray(
    A = [2.7,3.0, 0.0],    # [5,5, 2.0]
    ω = [5/8,4/8,6/8], #ω = [0.4,0.5,0.],
    ψ = [π/2, 0, 0],
    off = [-0.5,-1.4,6.7] # off
)

ComponentVector{Float64}(A = [2.7, 3.0, 0.0], ω = [0.625, 0.5, 0.75], ψ = [1.5707963267948966, 0.0, 0.0], off = [-0.5, -1.4, 6.7])

In [147]:
RefTrajLib.lissajous(0.0, trajectory_params_0)

3-element SVector{3, Float64} with indices SOneTo(3):
  2.2
 -1.4
  6.7

In [44]:
quad_LX = 0.08 # 0.08
quad_LY = 0.08

0.08

In [45]:
motor_pos = hcat(
    [
        [quad_LX, -quad_LY, 0.0],
        [-quad_LX, quad_LY, 0.0],
        [quad_LX, quad_LY, 0.0],
        [-quad_LX, -quad_LY, 0.0],
    ]...,
)

3×4 Matrix{Float64}:
  0.08  -0.08  0.08  -0.08
 -0.08   0.08  0.08  -0.08
  0.0    0.0   0.0    0.0

In [234]:
m = 0.5
g = 9.81
m*g

4.905

In [218]:
uhover = (m*g/4)*ones(4)

4-element Vector{Float64}:
 1.22625
 1.22625
 1.22625
 1.22625

In [219]:
DIhover = [0; 0; m*g]
# DIx0 = [0.; 0; 1.0; 0; 0; 0] # (pos_x, pos_y, pos_z, vel_x, vel_y, vel_z)
# DIumin = [-2.; -2; 0.8*m*g]
DIumax = [2.; 2; 2.0*m*g]

3-element Vector{Float64}:
 2.0
 2.0
 9.81

In [14]:
A = [0 0 0 1 0 0;0 0 0 0 1 0;0 0 0 0 0 1; 0 0 0 0 0 0; 0 0 0 0 0 0; 0 0 0 0 0 0.0]

6×6 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [15]:
B = [0. 0 0; 0 0 0;0 0 0; 1 0 0; 0 1 0; 0 0 1]

6×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [16]:
N = 40
Uref = [(g*m/4)*ones(4) for i = 1:(N-1)]

39-element Vector{Vector{Float64}}:
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 ⋮
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]
 [1.22625, 1.22625, 1.22625, 1.22625]

In [17]:
N = 40
DIUref = [[0; 0; m*g] for i = 1:(N-1)]

39-element Vector{Vector{Float64}}:
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 ⋮
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]
 [0.0, 0.0, 4.905]

In [18]:
using ComponentArrays, Plots, LinearAlgebra, Parameters
using Printf
using Rotations
using StaticArrays
using Parameters: @unpack

In [19]:
function hover_ω(params)
    @unpack k_f, mass, g = params
    return SVector{4}(sqrt(mass * g / (4 * k_f)) for i = 1:4)
end




function initialize_quad_params(quad_p)

    @unpack J, k_f, k_μ, motor_pos, motor_dir = quad_p

    invJ = inv(J)

    G = zeros(4, 4)

    Gf = k_f * ones(1, 4)

    GM = zeros(3, 4)
    for i = 1:4
        G[1, i] = k_f
        G[2:4, i] = cross(motor_pos[:, i], [0, 0, k_f])
        G[4, i] = -motor_dir[i] * k_μ
    end

    invG = inv(G)

    return ComponentArray(quad_p; invJ = invJ, invG = invG)
end
J = Diagonal([0.0023, 0.0023, 0.004])
quad_LX = 0.08
quad_LY = 0.08
const quadrotor_parameters =
    ComponentArray(
        mass = 1.0,            # mass of quadrotor [kg]
        J = diagm([0.0049, 0.0049, 0.0069]) |> collect,                # moment of inertia of quadrotor in body axes [kg/m^2]
        τ_m = 0.02,            # time constant for quadrotor motors [s]
        J_m = 6.62e-6,         # moment of inertia of each motor+propeller [kg/m^2]
        g = 9.81,              # acceleration due to gravity [m/s^2]
        k_f = 1.91e-6,         # thrust coefficient, [N / (rad/s)^2]
        k_μ = 2.7e-7,          # torque coefficient, [Nm / (rad/s)^2]
        k_drag_f = 0.1,        # linear drag coefficient, [N / (m/s)^2]
        k_drag_μ = 0.003 * I(3), # angular drag coefficient, [Nm / (rad/s)^2]
        ω_max = 5000.0,          # maximum rotation rate of motors [rad/s]
        motor_pos = hcat(
            [
                [quad_LX, -quad_LY, 0.0],
                [-quad_LX, quad_LY, 0.0],
                [quad_LX, quad_LY, 0.0],
                [-quad_LX, -quad_LY, 0.0],
            ]...,
        ),                        # motor positions in body-fixed frame (i-th column contains i-th motor pos)
        motor_dir = [1, 1, -1, -1], # motor directions in body-fixed frame
    ) |> initialize_quad_params

ComponentVector{Float64}(mass = 1.0, J = [0.0049 0.0 0.0; 0.0 0.0049 0.0; 0.0 0.0 0.0069], τ_m = 0.02, J_m = 6.62e-6, g = 9.81, k_f = 1.91e-6, k_μ = 2.7e-7, k_drag_f = 0.1, k_drag_μ = [0.003 0.0 0.0; 0.0 0.003 0.0; 0.0 0.0 0.003], ω_max = 5000.0, motor_pos = [0.08 -0.08 0.08 -0.08; -0.08 0.08 0.08 -0.08; 0.0 0.0 0.0 0.0], motor_dir = [1.0, 1.0, -1.0, -1.0], invJ = [204.08163265306123 0.0 0.0; 0.0 204.08163265306123 0.0; 0.0 0.0 144.92753623188406], invG = [130890.05235602095 -1.636125654450262e6 -1.636125654450262e6 -925925.9259259258; 130890.05235602094 1.636125654450262e6 1.636125654450262e6 -925925.9259259258; 130890.05235602095 1.6361256544502617e6 -1.6361256544502617e6 925925.925925926; 130890.05235602094 -1.636125654450262e6 1.636125654450262e6 925925.9259259258])

In [20]:
## Define the quad's initial condition
quad_ic = ComponentArray(
    x = zeros(3),
    v = zeros(3),
    R = 1.0(I(3)) |> collect, 
    Ω = zeros(3),
    ω = 2*hover_ω(quadrotor_parameters),
    soc = 60
);

In [20]:
quad_ic

ComponentVector{Float64}(x = [0.0, 0.0, 0.0], v = [0.0, 0.0, 0.0], R = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], Ω = [0.0, 0.0, 0.0], ω = [2266.3021983950557, 2266.3021983950557, 2266.3021983950557, 2266.3021983950557], soc = 60.0)

In [21]:
quad_n = [quad_ic, quad_ic, quad_ic]

3-element Vector{ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(x = 1:3, v = 4:6, R = ViewAxis(7:15, ShapedAxis((3, 3), NamedTuple())), Ω = 16:18, ω = 19:22, soc = 23)}}}}:
 ComponentVector{Float64}(x = [0.0, 0.0, 0.0], v = [0.0, 0.0, 0.0], R = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], Ω = [0.0, 0.0, 0.0], ω = [2266.3021983950557, 2266.3021983950557, 2266.3021983950557, 2266.3021983950557], soc = 60.0)
 ComponentVector{Float64}(x = [0.0, 0.0, 0.0], v = [0.0, 0.0, 0.0], R = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], Ω = [0.0, 0.0, 0.0], ω = [2266.3021983950557, 2266.3021983950557, 2266.3021983950557, 2266.3021983950557], soc = 60.0)
 ComponentVector{Float64}(x = [0.0, 0.0, 0.0], v = [0.0, 0.0, 0.0], R = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], Ω = [0.0, 0.0, 0.0], ω = [2266.3021983950557, 2266.3021983950557, 2266.3021983950557, 2266.3021983950557], soc = 60.0)

In [22]:
quad_n[1].x = [1; 2; 3]

3-element Vector{Int64}:
 1
 2
 3

In [24]:
quad_n[end].soc

60.0

In [ ]:
# This is how we will do the multi-robot quad 